# Homework 2: Word Similarity

Student Name: 

Student ID:

## General info

<b>Due date</b>: Thursday, 4 June 2020 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 10% of mark for class (with 9% on correctness + 1% on quality and efficiency of your code)

**Note**: As we will be implementing neural networks in this assignment, you're encouraged to build your notebook on **colab**. See the programming exercise in workshop-07 (`10-bert.ipynb`) if you are not familiar with colab.

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/17601/pages/using-jupyter-notebook-and-python?module_item_id=1678430) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. Deep learning libraries such as keras and pytorch are also allowed.  You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.




<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dataset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

### Question 1 (1.0 mark)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://alfonseca.org/eng/research/wordsim353.html">Similarity-353</a>. You need to first obtain this dataset, which is on Canvas (assignment 2). The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on **document frequencies** (not token frequencies) in the Brown corpus, in order to remove rare words. In this homework, **we will be treating the paragraphs of the Brown corpus as our "documents"**. You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized (lemmatize function provided). Store this preprocessed data in *brown_corpus* object (it will be used for question 4 and 5 later).

Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of **$< 8$** in this corpus. You should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*.

Note: the document frequency of a word denotes the number of documents that contains the word.

**Task**: Filter word pairs from *set1.tab* based on document frequencies. Produce *brown_corpus*, which is a list where each element is a set of words for one paragraph (e.g. the first element in *brown_corpus* should contain all the unique word types for the first paragraph). Produce *filtered_gold_standard*, a dictionary of filtered word pairs with human similarity ratings (the dictionary should have (word1, word2) as keys, and similarity ratings as values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *filtered_gold_standard*.

In [107]:
import csv
import nltk
import re
from nltk.corpus import brown
from nltk.corpus import wordnet as wn

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

# lemmatizer
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


###
# Your answer BEGINS HERE
###

# Read file into a dict
fil_dic = {}
with open('set1.tab', newline = '') as setab:                                                                                          
    reader = csv.reader(setab, delimiter='\t')
    next(reader, None)
    for game in reader:
        fil_dic[(game[0], game[1])] = float(game[2])


# Get alphabetical and lemmentized word into a dict for each paragraph
# Append dict of each paragraph
brown_corpus = []
fb = []

for para in brown.paras():
    tokens = []
    tk_set = set()
    for p in para:
        for token in p:
            # find all alphabetical unique expressions in the paragraph
            if token.isalpha() and token not in tk_set:
                tokens.append(lemmatize(token.lower()))
                tk_set.add(lemmatize(token.lower()))
                
    brown_corpus.append(tokens)
    
# Merge dicts in each paragraph into a large dict
merge_dic = {}
for tks in brown_corpus:
    for key in tks:
        try:
            merge_dic[key] += 1
        except:
            merge_dic[key] = 1
            
            
# Filter words pairs by the large dict
for k, v in fil_dic.items():
    try: 
        if merge_dic[k[0]] >= 8 and merge_dic[k[1]] >= 8:
            filtered_gold_standard[k] = v
    except:
        continue

        
print(len(filtered_gold_standard))
print(filtered_gold_standard)


[nltk_data] Downloading package brown to /Users/gey3/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/gey3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


94
{('love', 'sex'): 6.77, ('tiger', 'cat'): 7.35, ('tiger', 'tiger'): 10.0, ('book', 'paper'): 7.46, ('plane', 'car'): 5.77, ('train', 'car'): 6.31, ('telephone', 'communication'): 7.5, ('television', 'radio'): 6.77, ('drug', 'abuse'): 6.85, ('bread', 'butter'): 6.19, ('doctor', 'nurse'): 7.0, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('smart', 'student'): 4.62, ('smart', 'stupid'): 5.81, ('company', 'stock'): 7.08, ('stock', 'market'): 8.08, ('stock', 'phone'): 1.62, ('stock', 'egg'): 1.81, ('stock', 'live'): 3.73, ('stock', 'life'): 0.92, ('book', 'library'): 7.46, ('bank', 'money'): 8.12, ('wood', 'forest'): 7.73, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('holy', 'sex'): 1.62, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('tennis', 'racket'): 7.56, ('law', 'lawyer'): 8.38, ('movie', 'star'): 7.38, ('movie', 'critic'): 6.73, ('movie', 'theater'): 7.92, ('space', 'chemistry'): 4.88, ('alcohol', 'chemistry'): 

<b>For your testing: </b>

In [121]:
assert(len(brown_corpus)==15667)
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

### Question 2 (1.0 mark)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a **single primary sense**. We define single primary sense here as either: (a) having only one sense (i.e. only one synset), or (b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma.

Note: You should lowercase the lemmas of a synset when matching your word; and if there are multiple lowercased lemmas that match your word, you should sum up the count of all matching lemmas.

Additionally, you should remove any words where the primary sense is **not a noun** (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section.

Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria. When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*.

**Task**: Filter word pairs for any words which do not have a single primary sense and aren't nouns. Produce *final_gold_standard*, a dictionary of filtered word pairs with human similarity ratings. Note: this second filtering is applied on top of the first filtering (question 1). In other words, you shouldn't consider any word pairs that have already been discarded by the first filtering in this question.

**Check**: Use the assertion statements in *"For your testing"* for the expected *final_gold_standard*.

In [109]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}
word_primarysense = {} #a dictionary of (word, primary_sense) (used for next section); primary_sense is a synset

###
# Your answer BEGINS HERE
###

for k, v in filtered_gold_standard.items():
    
    # build a dictionary with synset and their count
    syns = wn.synsets(k[0])
    syn_dic = {}
    for syn in syns:
        count = 0
        for lemma in syn.lemmas():
            # filter all lemma with different name
            if lemma.name() == k[0]:
                count += lemma.count()
        syn_dic[syn] = count
    
    # sort all lemma and count pairs
    sort_dic1 = sorted(syn_dic.items(), key=lambda syn_dic: syn_dic[1], reverse=True)
    word_primarysense[k[0]] = sort_dic1[0][0]
    
    syns = wn.synsets(k[1])
    syn_dic = {}
    for syn in syns:
        count = 0
        for lemma in syn.lemmas():
            if lemma.name() == k[1]:
                count += lemma.count()
        syn_dic[syn] = count
        
    sort_dic2 = sorted(syn_dic.items(), key=lambda syn_dic: syn_dic[1], reverse=True)
    word_primarysense[k[1]] = sort_dic2[0][0]
    
    # find the one satisfying the requirement.
    if len(sort_dic1) > 1 and (sort_dic1[0][1] < 4*sort_dic1[1][1]):
        continue
    
    if len(sort_dic2) > 1 and (sort_dic2[0][1] < 4*sort_dic2[1][1]):
        continue
    
    final_gold_standard[k] = v

###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)

26
{('bread', 'butter'): 6.19, ('professor', 'doctor'): 6.62, ('student', 'professor'): 6.81, ('stock', 'egg'): 1.81, ('money', 'cash'): 9.08, ('king', 'queen'): 8.58, ('bishop', 'rabbi'): 6.69, ('football', 'basketball'): 6.81, ('football', 'tennis'): 6.63, ('alcohol', 'chemistry'): 5.54, ('baby', 'mother'): 7.85, ('car', 'automobile'): 8.94, ('journey', 'voyage'): 9.29, ('coast', 'shore'): 9.1, ('brother', 'monk'): 6.27, ('journey', 'car'): 5.85, ('coast', 'hill'): 4.38, ('forest', 'graveyard'): 1.85, ('monk', 'slave'): 0.92, ('coast', 'forest'): 3.15, ('psychology', 'doctor'): 6.42, ('psychology', 'mind'): 7.69, ('psychology', 'health'): 7.23, ('psychology', 'science'): 6.71, ('planet', 'moon'): 8.08, ('planet', 'galaxy'): 8.11}


<b>For your testing:</b>

In [110]:
# assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Computing word similiarity with Lin similarity, NPMI and LSA (3 marks)

### Question 3 (1.0 mark)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*.

**Task**: Compute word pair similarity using Lin similarity for the test set. Produce *lin_similarities*, a dictionary of word pairs (keys) and computed Lin similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *lin_similarities*. 

In [111]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###

semcor_ic = wordnet_ic.ic('ic-semcor.dat')

for pair in final_gold_standard:
    syn_name1 = word_primarysense[pair[0]].name()
    syn_name2 = word_primarysense[pair[1]].name()
    sim_val = wn.synset(syn_name1).lin_similarity(wn.synset(syn_name2),semcor_ic)
    lin_similarities[pair] = sim_val

###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to /Users/gey3/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


{('bread', 'butter'): 0.726992234769291, ('professor', 'doctor'): 0.7770369763910114, ('student', 'professor'): 0.23953150362674094, ('stock', 'egg'): -0.0, ('money', 'cash'): 0.8137879336398085, ('king', 'queen'): 0.2046706224770807, ('bishop', 'rabbi'): 0.7522246770216547, ('football', 'basketball'): 0.7704674954719576, ('football', 'tennis'): 0.7307921720649426, ('alcohol', 'chemistry'): 0.09135314199193188, ('baby', 'mother'): 0.654948986686927, ('car', 'automobile'): 1.0, ('journey', 'voyage'): 0.8276818213211407, ('coast', 'shore'): 0.9632037579820005, ('brother', 'monk'): 0.20788439871426262, ('journey', 'car'): -0.0, ('coast', 'hill'): 0.7285879249527557, ('forest', 'graveyard'): -0.0, ('monk', 'slave'): 0.20112728495581914, ('coast', 'forest'): -0.0, ('psychology', 'doctor'): -0.0, ('psychology', 'mind'): 0.3301492601041217, ('psychology', 'health'): 0.08338132296510331, ('psychology', 'science'): 0.8124859588519519, ('planet', 'moon'): 0.12715707066005769, ('planet', 'galaxy'

<b>For your testing:</b>

In [112]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

### Question 4 (1.0 mark)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-log_2(p(x,y))} = \frac{log_2(p(x)p(y))}{log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NPMI is -1. NPMI is normalized between [-1, +1]. 

You should use the *brown_corpus* object you've created in question 1 as your corpus here. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*.

**Task**: Compute word pair similarity using NPMI similarity for the test set. Produce *NPMI_similarities*, a dictionary of word pairs (keys) and computed NPMI similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *NPMI_similarities*.

In [113]:
import math

# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###

word_num = 0
for para in brown_corpus:
    word_num += len(para)

for pair in final_gold_standard:
    Nx = 0
    Ny = 0
    Nxy = 0
    
    for para in brown_corpus:
        if pair[0] in para:
            Nx += 1
        if pair[1] in para:
            Ny += 1
        if pair[0] in para and pair[1] in para:
            Nxy += 1
            
    Px = Nx/word_num
    Py = Ny/word_num
    Pxy = Nxy/word_num
            
    if Nxy == 0: 
        NPMI = -1
    else: 
        NPMI = math.log2(Px*Py) / math.log2(Pxy) - 1
        
    NPMI_similarities[pair] = NPMI

###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('bread', 'butter'): 0.6560844757016677, ('professor', 'doctor'): -1, ('student', 'professor'): 0.5398401080836155, ('stock', 'egg'): 0.3734592156434495, ('money', 'cash'): 0.449670426983076, ('king', 'queen'): 0.42269832732999113, ('bishop', 'rabbi'): -1, ('football', 'basketball'): 0.7186188959099991, ('football', 'tennis'): -1, ('alcohol', 'chemistry'): 0.6275778417305191, ('baby', 'mother'): 0.5193733276364185, ('car', 'automobile'): 0.5473471991967662, ('journey', 'voyage'): -1, ('coast', 'shore'): 0.5898305160711814, ('brother', 'monk'): 0.4343970924690428, ('journey', 'car'): -1, ('coast', 'hill'): 0.3444853504442853, ('forest', 'graveyard'): -1, ('monk', 'slave'): -1, ('coast', 'forest'): 0.4623497256519702, ('psychology', 'doctor'): 0.465547476731216, ('psychology', 'mind'): 0.45073901608292877, ('psychology', 'health'): -1, ('psychology', 'science'): 0.594361940775401, ('planet', 'moon'): 0.6610412684504505, ('planet', 'galaxy'): -1}


<b>For your testing:</b>

In [114]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

### Question 5 (1.0 mark)

**Instructions:** Here we'll use singular value decomposition (SVD) to derive similarity scores using the Latent Semantic Analysis (LSA) method. Recall that LSA applies SVD and truncation to get a dense vector representation of a word type. To measure similarity between two words we calculate cosine similarity between the LSA vectors of the words.

We'll first build a term-document frequency matrix (as before, a document is a paragraph in Brown corpus). That is, the rows corresponds to words in the vocabulary, and the columns represent the paragraphs/documents in Brown corpus. Each cell records the document frequency of a word (1 if the word appears in the document, 0 otherwise). You should use the *brown_corpus* object created in question 1 as your corpus to build the term-document frequency matrix.

Given the term-document frequency matrix, we'll use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for the word pairs in the test set.

You can use the workshop's notebook on distributional similarity (``09-distributional-similarity.ipynb``) as a starting point, but note that we use term-document frequency matrix (as opposed to the tf-idf matrix in the workshop).

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*.

**Task**: Compute word pair similarity using LSA. Produce *LSA_similarities*, a dictionary of word pairs (keys) and computed LSA similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *LSA_similarities*. 



In [115]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from numpy.linalg import norm
from sklearn.feature_extraction import DictVectorizer

# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###

# Referenced from tute 9 notebook
def get_BOW(text):
    BOW = {}
    for word in text:
        # only record the occurrance
        BOW[word.lower()] = 1
    return BOW

texts = []
for para in brown_corpus:
    texts.append(get_BOW(para))

#  Build LSA Matrix
vectorizer = DictVectorizer()
LSA_matrix = vectorizer.fit_transform(texts)

# Manually build LSA
# voc = {}
# index = 0
# for doc in brown_corpus:
#     for word in doc:
#         if word not in voc:
#             voc[word] = index
#             index += 1
            
# vocab_len = len(voc)
# print(vocab_len)

# document_len = len(brown_corpus)

# LSA_matrix = np.zeros((document_len, vocab_len))

# for ind_doc in range(document_len):
#     for word in doc:
#         LSA_matrix[ind_doc][voc[word]] += 1 

# for ind_doc in range(document_len):
#     for word in doc:
#         if LSA_matrix[ind_doc][voc[word]] > 1:
#             print(ind_doc, word, LSA_matrix[ind_doc][voc[word]])

print(LSA_matrix.shape)

# Build SVD and get low rank matrix   
brown_matrix_transposed = csr_matrix(LSA_matrix).transpose()

svd = TruncatedSVD(n_components=500)
brown_matrix_lowrank = svd.fit_transform(brown_matrix_transposed)

# calculate cosine similarity
def cos_sim(a, b):
    return np.dot(a, b)/(norm(a)*norm(b))


# Get the vector for words and get cosine similarity
for pair in final_gold_standard:
    v1 = brown_matrix_lowrank[vectorizer.vocabulary_[pair[0]]]
    v2 = brown_matrix_lowrank[vectorizer.vocabulary_[pair[1]]]
    LSA_similarities[pair] = cos_sim(v1, v2)

###
# Your answer ENDS HERE
###

print(LSA_similarities)


(15667, 29130)
{('bread', 'butter'): 0.3094160278950983, ('professor', 'doctor'): 0.055409230501290964, ('student', 'professor'): 0.2736874930689444, ('stock', 'egg'): 0.141773212683901, ('money', 'cash'): 0.14553634116809724, ('king', 'queen'): 0.09872337018743879, ('bishop', 'rabbi'): 0.004503034305340192, ('football', 'basketball'): 0.23766608200017852, ('football', 'tennis'): 0.12956464186163236, ('alcohol', 'chemistry'): 0.10223156007047768, ('baby', 'mother'): 0.36730522585623454, ('car', 'automobile'): 0.3444892685393773, ('journey', 'voyage'): 0.08057642016451437, ('coast', 'shore'): 0.40794096562682586, ('brother', 'monk'): 0.09206735390342959, ('journey', 'car'): 0.018600976669799308, ('coast', 'hill'): 0.18594045901746686, ('forest', 'graveyard'): 0.03508736282155306, ('monk', 'slave'): -0.014737800086650795, ('coast', 'forest'): 0.14289370537746357, ('psychology', 'doctor'): 0.12286147222870701, ('psychology', 'mind'): 0.12948106473328452, ('psychology', 'health'): 0.056023

<b>For your testing:</b>

In [116]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## Computing word similarity with feedforward language model (3 marks)

### Question 6 (1.0 mark)

**Instructions**: Here we'll build a n-gram neural language model to learn word embeddings, and compute word similarity based on the word embeddings. As before we will use the Brown corpus as training data for our language model, and the first step is to collect the vocabulary.

As before, we'll treat paragraphs in the Brown corpus as our documents. The first 12K paragraphs/documents will serve as our training data, and the rest (3K+ documents) as development data. The first step towards building a language model is to collect the vocabulary, i.e. the set of unique word types in our training data. When collecting the word types, you should lowercase all words, and only keep word types that have a frequency $>= 5$. Store your vocabulary in the _vocab_ object.

Note: 
  - we'll be using _$<$UNK$>$_ to represent unseen words, and so _vocab_ is initialised with the special _$<$UNK$>$_.
  - you do not need to do any other additional preprocessing aside from the aforementioned steps (e.g. no need to remove symbols, etc)
  - you should not use *brown_corpus* here, as you need the words in their original order (*brown_corpus* stores a set of words for each paragraph, and so do not contain word order information)

**Task**: Collect a set of unique word types in the training portion (first 12K paragraphs) of the Brown corpus. Produce _vocab_, which is a set that contains all the word types.

**Check**: Use the assertion statements in *"For your testing"* below to check the vocabulary size.

In [117]:
num_train = 12000
UNK_symbol = "<UNK>"
vocab = set([UNK_symbol])

###
# Your answer BEGINS HERE
###

tf = {}
for para in brown.paras()[:num_train]:
    for sent in para:
        for word in sent:
            lw = word.lower()
            if tf.get(lw, -1) != -1:
                tf[lw] += 1
                if tf[lw] >= 5: 
                    vocab.add(lw)
            else:
                tf[lw] = 1

###
# Your answer ENDS HERE
###

print(len(vocab))

12681


**For your testing:**

In [118]:
assert(len(vocab) > 8000 and len(vocab) < 20000)

### Question 7 (1.0 mark)

**Instructions**: As we'll be building a trigram neural language model (based on lecture 7, page 20), the next step is to collect trigrams to construct our training data. In a trigram neural language model, for example if we have the trigram _cow eats grass_, the input to the model is the first two terms of a trigram (_cow_ and _eats_), and the language model's aim is to predict the last term of the trigram (_grass_). Your task here is to construct the training and development data for the language model. Just like the previous step, the first 12K paragraphs will serve as our training data, and the remaining 3K+ will be for development. You'll need to map words into IDs when constructing the training and development data. Any words that are not in _vocab_ should be mapped to the special _$<$UNK$>$_ symbol.

As an example, given the sentence "_a big fat hungry cow ._", you should create the following training examples:

|input|target|
|:---:|:----:|
|<i>a</i>, _big_|_fat_|
|_big_, _fat_|_hungry_|
|_fat_, _hungry_|_cow_|
|_hungry_, _cow_|_._|


Note:
   - _vocab_ is a set and so does not map words into IDs. You'll need to create a word to ID mapping first based on _vocab_.
   - A trigram should not cross sentence boundary.
   - You should ignore sentences that have less than 3 words (as they are too short to form trigrams).
   - We won't need special starting symbol when collecting the trigrams, as we are only interested in learning word embeddings here (and not computing probabilities of a sentence).


**Task**: Create training and development data. The training input and target data should be stored in the <i>x_train</i> and <i>y_train</i> respectively, and development input and target data in <i>x_dev</i> and <i>y_dev</i> respectively.

**Check**: Use the assertion statements in *"For your testing"* below for the expected shape of the outputs.

In [119]:
###
# Your answer BEGINS HERE
###

# add id to form vocab_dic
vocab_dic = {}
for id, word in enumerate(vocab):
    vocab_dic[word] = id

# get id by filtering unknown and known words
def get_id(word):
    if word.lower() not in vocab:
        return vocab_dic[UNK_symbol]
    else:
        return vocab_dic[word.lower()]

# Get trigram input and ouput in lists
def gen_data(documents):
    X = []
    Y = []
    count = 0
    for para in documents:
        for sentence in para:
            if len(sentence) < 3:
                continue
            for index in range(len(sentence) - 2):
                x1 = get_id(sentence[index])
                x2 = get_id(sentence[index+1])
                y  = get_id(sentence[index+2])
                X.append([x1, x2])
                Y.append(y)
    return X, Y
            
# Get training and developing data
xt, yt  = gen_data(brown.paras()[:num_train])
xd, yd  = gen_data(brown.paras()[num_train:])

x_train = np.array(xt) 
y_train = np.array(yt)
x_dev   = np.array(xd) 
y_dev   = np.array(yd)
        
###
# Your answer ENDS HERE
###

print(x_train.shape)
print(y_train.shape)
print(x_dev.shape)
print(y_dev.shape)

(872823, 2)
(872823,)
(174016, 2)
(174016,)


**For your testing:**

In [120]:
assert(x_train.shape[0] == y_train.shape[0])
assert(x_dev.shape[0] == y_dev.shape[0])
assert(x_train.shape[0] > 500000)
assert(x_dev.shape[0] > 50000)

### Question 8 (1.0 mark)

**Instructions**: Now let's build the trigram neural language model. We'll use the language model described in lecture 7 (page 20):

$x' = e(x_1) \oplus e(x_2)$

$h = \tanh(W_1 x' + b)$

$y = $ softmax$(W_2 h)$

where $\oplus$ is the concatenation operation, $x_1$ and $x_2$ are the input words, $e$ is an embedding function, and $y$ is the target word. You can use either `keras` or `pytorch` for building your model.

Set the dimension of the word embeddings and $h$ to 100, and train your model with 3 epochs with a batch size of 256. You will not need to tune hyper-parameters for this task.

After the model is trained, use the word embeddings to compute word similarity in the test set. Store the similarity scores in a dictionary called *lm_similarities*.

Note:
  - For words in the test set that are not in your vocabulary, you should treat them as unknown words. In other words, you should use $<$UNK$>$'s embedding to represent these words.
  - The training may take some time on CPU. You can run your notebook on colab with a GPU if you want faster training (see the programming exercise in workshop-07 if you're not familiar with colab)

**Task**: Train a trigram neural language model, and use the learned word embeddings to compute word similarity. Produce *lm_similarities*, a dictionary that contains word pairs as keys and similarity scores as values.

In [123]:
###
# Your answer BEGINS HERE
###

from keras.models import Sequential
from keras import layers

lm_similarities = {}

embedding_dim = 100

#model definition
model = Sequential()
model.add(layers.Embedding(input_dim=x_train.shape[0], 
                           output_dim=embedding_dim,
                           input_length=2))
model.add(layers.Flatten())
model.add(layers.Dense(100, input_dim=x_train.shape[0], activation='tanh'))
model.add(layers.Dense(1, activation='softmax'))

#since it's a binary classification problem, we use a binary cross entropy loss here
model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()

#training
model.fit(x_train, y_train, epochs=3, verbose=True, validation_data=(x_dev, y_dev), batch_size=256)

loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("\nTesting Accuracy:  {:.4f}".format(accuracy))


# embeddings = model.get_layer(index=0).get_weights()[0] #word embeddings layer

# emb_love = embeddings[tokenizer.word_index["love"]] #embeddings for 'love'
# emb_like = embeddings[tokenizer.word_index["like"]]
# emb_lukewarm = embeddings[tokenizer.word_index["lukewarm"]]
# emb_bad = embeddings[tokenizer.word_index["bad"]]








###
# Your answer ENDS HERE
###


print(lm_similarities)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 100)            87282300  
_________________________________________________________________
flatten_1 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 87,302,501
Trainable params: 87,302,501
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 872823 samples, validate on 174016 samples
Epoch 1/3
 60928/872823 [=>............................] - ETA: 1:00:37 - loss: -92274.1733 -

KeyboardInterrupt: 

## 3. Comparison with the Gold Standard (1 mark)

### Question 9 (1.0 mark)

**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA', 'lm').

**Task**: Compute the Pearson correlation coefficient between the estimated similarity scores (Lin, NPMI and LSA similarities) and the gold standard similarity ratings. Produce *pearson_correlations*, a dictionary containing the methods as keys and correlations as values.

**Check**: Use the assertion statements in *"For your testing"* below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs! 

In [ ]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA', 'lm'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###

pearson_correlations



###
# Your answer ENDS HERE
###

print(pearson_correlations)

<b>For your testing:</b>

In [ ]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.